In [1]:
import pandas as pd
# import numpy as np
pd.set_option("max_columns", None) # all df cols

from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder, StandardScaler
# from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
# from sklearn.compose import ColumnTransformer, make_column_transformer
# from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

## Step 1: Prepare

### Get team name

In [2]:
dict_name = {}
with open('full_name.txt', 'r') as f:
    for line in f:
        name = line.split(',') # [short_name, full_name]
        dict_name[name[0]] = name[1].strip()
dict_name

{'ATL': 'Atlanta Hawks',
 'BRK': 'Brooklyn Nets',
 'BOS': 'Boston Celtics',
 'CHO': 'Charlotte Hornets',
 'CHI': 'Chicago Bulls',
 'CLE': 'Cleveland Cavaliers',
 'DAL': 'Dallas Mavericks',
 'DEN': 'Denver Nuggets',
 'DET': 'Detroit Pistons',
 'GSW': 'Golden State Warriors',
 'HOU': 'Houston Rockets',
 'IND': 'Indiana Pacers',
 'LAC': 'Los Angeles Clippers',
 'LAL': 'Los Angeles Lakers',
 'MEM': 'Memphis Grizzlies',
 'MIA': 'Miami Heat',
 'MIL': 'Milwaukee Bucks',
 'MIN': 'Minnesota Timberwolves',
 'NOP': 'New Orleans Pelicans',
 'NYK': 'New York Knicks',
 'OKC': 'Oklahoma City Thunder',
 'ORL': 'Orlando Magic',
 'PHI': 'Philadelphia 76ers',
 'PHO': 'Phoenix Suns',
 'POR': 'Portland Trail Blazers',
 'SAC': 'Sacramento Kings',
 'SAS': 'San Antonio Spurs',
 'TOR': 'Toronto Raptors',
 'UTA': 'Utah Jazz',
 'WAS': 'Washington Wizards'}

### Get column description

In [3]:
lst_description = []
with open('description.txt', 'r') as f:
    for line in f:
        lst_description.append(line.strip())
lst_description

['Date : Date',
 'H_Rk : Home Rank',
 'H_Team : Home Team',
 'H_G : Home Games',
 'H_MP : Home Minutes Played',
 'H_FG : Home Field Goals',
 'H_FGA : Home Field Goal Attempts',
 'H_FG% : Home Field Goal Percentage',
 'H_3P : Home 3-Point Field Goals',
 'H_3PA : Home 3-Point Field Goal Attempts',
 'H_3P% : Home 3-Point Field Goal Percentage',
 'H_2P : Home 2-Point Field Goals',
 'H_2PA : Home 2-point Field Goal Attempts',
 'H_2P% : Home 2-Point Field Goal Percentage',
 'H_FT : Home Free Throws',
 'H_FTA : Home Free Throw Attempts',
 'H_FT% : Home Free Throw Percentage',
 'H_ORB : Home Offensive Rebounds',
 'H_DRB : Home Defensive Rebounds',
 'H_TRB : Home Total Rebounds',
 'H_AST : Home Assists',
 'H_STL : Home Steals',
 'H_BLK : Home Blocks',
 'H_TOV : Home Turnovers',
 'H_PF : Home Personal Fouls',
 'H_PTS : Home Points',
 'A_Rk : Away Rank',
 'A_Team : Away Team',
 'A_G : Away Games',
 'A_MP : Away Minutes Played',
 'A_FG : Away Field Goals',
 'A_FGA : Away Field Goal Attempts',
 'A_

### Get preseason stats

In [4]:
pre_stat_df = pd.read_csv('preseason_data.csv')
pre_stat_df.head()

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1.0,Golden State Warriors,82,240.6,42.8,85.1,0.503,11.3,28.9,0.391,31.5,56.2,0.560,16.6,20.3,0.815,8.4,35.1,43.5,29.3,8.0,7.5,15.5,19.6,113.5
1,2.0,Houston Rockets,82,240.9,38.7,84.2,0.460,15.3,42.3,0.362,23.4,41.9,0.558,19.6,25.1,0.781,9.0,34.5,43.5,21.5,8.5,4.8,13.8,19.5,112.4
2,3.0,New Orleans Pelicans,82,243.4,42.7,88.3,0.483,10.2,28.2,0.362,32.5,60.1,0.540,16.1,20.9,0.772,8.7,35.7,44.3,26.8,8.0,5.9,14.9,19.1,111.7
3,4.0,Toronto Raptors,82,241.8,41.3,87.4,0.472,11.8,33.0,0.358,29.5,54.4,0.541,17.3,21.8,0.794,9.8,34.2,44.0,24.3,7.6,6.1,13.4,21.7,111.7
4,5.0,Cleveland Cavaliers,82,240.6,40.4,84.8,0.476,12.0,32.1,0.372,28.4,52.6,0.540,18.1,23.3,0.779,8.5,33.7,42.1,23.4,7.1,3.8,13.7,18.6,110.9


1. Drop unnecessary cols:
- Ranking: 
- Game: const 82
- '%': calculate
- 'PTS: 
2. Rename:

In [5]:
def getShortName(input_name):    
    for short_name, full_name in dict_name.items():
        if input_name == full_name:
            return short_name

pre_stat_df.drop(['Rk', 'G', 'MP', 'FG','FGA','FG%', '3P%', '2P%', 'FT%', 'PTS'], axis=1, inplace=True)
pre_stat_df['Team'] = [getShortName(name) for name in pre_stat_df['Team']]
pre_stat_df.head()

,Team,3P,3PA,2P,2PA,FT,FTA,ORB,DRB,TRB,AST,STL,BLK,TOV,PF
0,GSW,11.3,28.9,31.5,56.2,16.6,20.3,8.4,35.1,43.5,29.3,8.0,7.5,15.5,19.6
1,HOU,15.3,42.3,23.4,41.9,19.6,25.1,9.0,34.5,43.5,21.5,8.5,4.8,13.8,19.5
2,NOP,10.2,28.2,32.5,60.1,16.1,20.9,8.7,35.7,44.3,26.8,8.0,5.9,14.9,19.1
3,TOR,11.8,33.0,29.5,54.4,17.3,21.8,9.8,34.2,44.0,24.3,7.6,6.1,13.4,21.7
4,CLE,12.0,32.1,28.4,52.6,18.1,23.3,8.5,33.7,42.1,23.4,7.1,3.8,13.7,18.6


In [6]:
# get col name
col_name = list(pre_stat_df.columns)
col_name

['Team',
 '3P',
 '3PA',
 '2P',
 '2PA',
 'FT',
 'FTA',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF']

### Get preseason elo

In [7]:
pre_elo_df = pd.read_csv('preseason_elo.csv').set_index('Name')
pre_elo_df.head()

,Elo
Name,
TOR,1805
GSW,1863
MIL,1801
POR,1662
HOU,1773


## Step 2: raw_data (from web scraping)

In [8]:
raw_df = pd.read_csv('raw_data.csv')
raw_df.head()

,Date,H_Team,H_MP,H_FG,H_FGA,H_FG%,H_3P,H_3PA,H_3P%,H_FT,H_FTA,H_FT%,H_ORB,H_DRB,H_TRB,H_AST,H_STL,H_BLK,H_TOV,H_PF,H_PTS,H_+/-,A_Team,A_MP,A_FG,A_FGA,A_FG%,A_3P,A_3PA,A_3P%,A_FT,A_FTA,A_FT%,A_ORB,A_DRB,A_TRB,A_AST,A_STL,A_BLK,A_TOV,A_PF,A_PTS,A_+/-
0,"July 20, 2021",MIL,240,37,82,0.451,6,27,0.222,25,29,0.862,11,42,53,20,10,6,18,17,105,NaN,PHO,240,38,86,0.442,6,25,0.240,16,19,0.842,6,31,37,14,11,4,14,21,98,NaN
1,"July 17, 2021",PHO,240,48,87,0.552,13,19,0.684,10,11,0.909,8,27,35,23,9,5,8,20,119,NaN,MIL,240,50,87,0.575,14,28,0.500,9,17,0.529,11,26,37,26,7,1,11,17,123,NaN
2,"July 14, 2021",MIL,240,39,97,0.402,7,29,0.241,24,29,0.828,17,31,48,22,11,3,5,18,109,NaN,PHO,240,40,78,0.513,7,23,0.304,16,19,0.842,5,35,40,18,3,9,17,24,103,NaN
3,"July 11, 2021",MIL,240,43,90,0.478,14,36,0.389,20,26,0.769,13,34,47,28,8,2,9,18,120,NaN,PHO,240,40,83,0.482,9,31,0.290,11,16,0.688,6,30,36,21,3,3,14,24,100,NaN
4,"July 8, 2021",PHO,240,43,88,0.489,20,40,0.500,12,14,0.857,11,32,43,26,7,2,12,19,118,NaN,MIL,240,42,93,0.452,9,31,0.290,15,23,0.652,18,28,46,21,8,7,9,17,108,NaN


In [9]:
raw_df.shape

(3626, 43)

In [10]:
# no nead % => drop
raw_df.drop(['H_FG%', 'H_3P%', 'H_FT%','A_FG%', 'A_3P%', 'A_FT%'], axis=1, inplace=True)

In [11]:
# replace ['H_PTS', 'A_PTS'] by ['H_Dif']
raw_df['H_Dif'] = raw_df.apply(lambda x: x['H_PTS'] - x['A_PTS'], axis=1)
raw_df.drop(['H_PTS', 'A_PTS'], axis = 1, inplace = True)
raw_df['H_Dif']

0        7
1       -4
2        6
3       20
4       10
        ..
3621    28
3622     3
3623    -1
3624     8
3625    18
Name: H_Dif, Length: 3626, dtype: int64

In [12]:
raw_df.dtypes

Date       object
H_Team     object
H_MP        int64
H_FG        int64
H_FGA       int64
H_3P        int64
H_3PA       int64
H_FT        int64
H_FTA       int64
H_ORB       int64
H_DRB       int64
H_TRB       int64
H_AST       int64
H_STL       int64
H_BLK       int64
H_TOV       int64
H_PF        int64
H_+/-     float64
A_Team     object
A_MP        int64
A_FG        int64
A_FGA       int64
A_3P        int64
A_3PA       int64
A_FT        int64
A_FTA       int64
A_ORB       int64
A_DRB       int64
A_TRB       int64
A_AST       int64
A_STL       int64
A_BLK       int64
A_TOV       int64
A_PF        int64
A_+/-     float64
H_Dif       int64
dtype: object

In [13]:
num_cols = list(raw_df.select_dtypes(include=['int64', 'float64']).columns)
df = raw_df[num_cols]
def missing_percentage(c):
    return (c.isna().mean() * 100).round(1)
def median(c):
    return c.quantile(0.5).round(1)
df.agg([missing_percentage, 'min', median, 'max'])

,H_MP,H_FG,H_FGA,H_3P,H_3PA,H_FT,H_FTA,H_ORB,H_DRB,H_TRB,H_AST,H_STL,H_BLK,H_TOV,H_PF,H_+/-,A_MP,A_FG,A_FGA,A_3P,A_3PA,A_FT,A_FTA,A_ORB,A_DRB,A_TRB,A_AST,A_STL,A_BLK,A_TOV,A_PF,A_+/-,H_Dif
missing_percentage,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0
min,240.0,21.0,64.0,2.0,10.0,2.0,4.0,1.0,18.0,22.0,10.0,0.0,0.0,1.0,6.0,NaN,240.0,25.0,63.0,2.0,12.0,3.0,4.0,0.0,19.0,24.0,10.0,0.0,0.0,2.0,7.0,NaN,-57.0
median,240.0,41.0,88.0,12.0,33.0,17.0,23.0,10.0,35.0,45.0,25.0,7.0,5.0,13.0,20.0,NaN,240.0,41.0,88.0,12.0,33.0,17.0,22.0,10.0,34.0,44.0,24.0,7.0,5.0,13.0,21.0,NaN,3.0
max,340.0,63.0,124.0,28.0,70.0,44.0,52.0,25.0,55.0,70.0,50.0,22.0,20.0,28.0,38.0,NaN,340.0,60.0,119.0,29.0,63.0,41.0,54.0,26.0,55.0,71.0,41.0,19.0,14.0,29.0,35.0,NaN,53.0


In [14]:
# '+/-' missing 100% => drop
raw_df.drop(['H_+/-', 'A_+/-'], axis = 1, inplace = True)

In [15]:
# 'MP' median 240 => drop
raw_df.drop(['H_MP', 'A_MP'], axis = 1, inplace = True)

In [16]:
# Field Goal => 3P & 2P
raw_df['H_2P'] = raw_df.apply(lambda x: x['H_FG'] - x['H_3P'], axis=1)
raw_df['H_2PA'] = raw_df.apply(lambda x: x['H_FGA'] - x['H_3PA'], axis=1)
raw_df['A_2P'] = raw_df.apply(lambda x: x['A_FG'] - x['A_3P'], axis=1)
raw_df['A_2PA'] = raw_df.apply(lambda x: x['A_FGA'] - x['A_3PA'], axis=1)
raw_df.drop(['H_FG', 'H_FGA', 'A_FG', 'A_FGA'], axis=1, inplace=True)

In [17]:
lst_H = ['H_'+col for col in col_name]
lst_A = ['A_'+col for col in col_name]

In [18]:
# reorder new 2P cols
lst_new_cols = ['Date'] + lst_H + lst_A + ['H_Dif']
raw_df = raw_df[lst_new_cols]
raw_df.head()

,Date,H_Team,H_3P,H_3PA,H_2P,H_2PA,H_FT,H_FTA,H_ORB,H_DRB,H_TRB,H_AST,H_STL,H_BLK,H_TOV,H_PF,A_Team,A_3P,A_3PA,A_2P,A_2PA,A_FT,A_FTA,A_ORB,A_DRB,A_TRB,A_AST,A_STL,A_BLK,A_TOV,A_PF,H_Dif
0,"July 20, 2021",MIL,6,27,31,55,25,29,11,42,53,20,10,6,18,17,PHO,6,25,32,61,16,19,6,31,37,14,11,4,14,21,7
1,"July 17, 2021",PHO,13,19,35,68,10,11,8,27,35,23,9,5,8,20,MIL,14,28,36,59,9,17,11,26,37,26,7,1,11,17,-4
2,"July 14, 2021",MIL,7,29,32,68,24,29,17,31,48,22,11,3,5,18,PHO,7,23,33,55,16,19,5,35,40,18,3,9,17,24,6
3,"July 11, 2021",MIL,14,36,29,54,20,26,13,34,47,28,8,2,9,18,PHO,9,31,31,52,11,16,6,30,36,21,3,3,14,24,20
4,"July 8, 2021",PHO,20,40,23,48,12,14,11,32,43,26,7,2,12,19,MIL,9,31,33,62,15,23,18,28,46,21,8,7,9,17,10


### Step 2: Create dataframe

In [19]:
def median(c):
    return c.quantile(0.5)

def get_recent(df, index, team_name, num_recent_games = 10):
    data = []
    
    while(len(data) < num_recent_games and index > 0):
        index -= 1
        if (team_name == df.iloc[index]['H_Team']):
            data.append(list(df.iloc[index][lst_H]))
        elif (team_name == df.iloc[index]['A_Team']):
            data.append(list(df.iloc[index][lst_A]))
            
    if index < num_recent_games:
        data.append(list(pre_stat_df.loc[pre_stat_df['Team'] == team_name].iloc[0]))
            
    return pd.DataFrame(data, columns = col_name).agg([median]).values.tolist()[0]

In [20]:
raw_df.iloc[12]

Date      June 26, 2021
H_Team              LAC
H_3P                  5
H_3PA                31
H_2P                 22
H_2PA                52
H_FT                 21
H_FTA                32
H_ORB                15
H_DRB                36
H_TRB                51
H_AST                13
H_STL                 4
H_BLK                 2
H_TOV                10
H_PF                 19
A_Team              PHO
A_3P                  4
A_3PA                20
A_2P                 27
A_2PA                66
A_FT                 18
A_FTA                21
A_ORB                17
A_DRB                42
A_TRB                59
A_AST                16
A_STL                 4
A_BLK                 8
A_TOV                14
A_PF                 27
H_Dif                -4
Name: 12, dtype: object

In [21]:
raw_df.iloc[90]

Date      May 18, 2021
H_Team             IND
H_3P                16
H_3PA               35
H_2P                42
H_2PA               70
H_FT                12
H_FTA               15
H_ORB               12
H_DRB               42
H_TRB               54
H_AST               35
H_STL                7
H_BLK                5
H_TOV               16
H_PF                21
A_Team             CHO
A_3P                12
A_3PA               40
A_2P                31
A_2PA               54
A_FT                19
A_FTA               25
A_ORB                7
A_DRB               29
A_TRB               36
A_AST               26
A_STL                8
A_BLK                6
A_TOV               14
A_PF                14
H_Dif               27
Name: 90, dtype: object

In [23]:
data = []

for index in range(len(raw_df.index)):
    row = [raw_df.iloc[index]['Date']]
    
    H_name = raw_df.iloc[index]['H_Team']
    row += [H_name]
    row += get_recent(raw_df, index, H_name)
    
    A_name = raw_df.iloc[index]['A_Team']
    row += [A_name]
    row += get_recent(raw_df, index, A_name)
    
    row += [raw_df.iloc[index]['H_Dif']]
    data.append(row)

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(data, columns=lst_new_cols)
df.head()

In [ ]:
df.to_csv('data.csv', encoding='utf-8', index=False)